In [36]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
def obr(df):
    df=df.drop(["id_contract","id_client","BIRTHDATE"], axis=1)
    columns_sum_LAG = [i for i in data_train.columns if i[0]=="F" and len(i)>=5]
    columns_sum = [i for i in data_train.columns if i[0]=="F" and len(i)==5]
    for i in columns_sum:
        df[i].fillna(0, inplace=True)
        df['SUM_F'] = data_train.loc[:, "F1100":"F2410"].sum(axis=1)
        df['SUM_F_LAGS'] = data_train.loc[:, "F1150_LAG1":"F2400_LAG1"].sum(axis=1)
    return df
data_train = pd.read_csv('train_dataset_hackathon_mkb.csv',sep = ';')
desc = data_train.describe()
data_test = pd.read_csv('test_dataset_hackathon_mkb.csv',sep = ';')
data_train['WORKERSRANGE_1'] = data_train['WORKERSRANGE']
data_test['WORKERSRANGE_1'] = data_test['WORKERSRANGE']
category_f = ['OKFS_GROUP','OKOPF_GROUP',
                    'OKATO_FED','EGRPOINCLUDED','WORKERSRANGE_1']


def make_null_features(data):
     
    data['is_null_0-10'] = data.iloc[:, 0:10].isnull().sum(axis=1)
    data['is_null_10-20'] = data.iloc[:, 10:20].isnull().sum(axis=1)
    data['is_null_20-30'] = data.iloc[:, 20:30].isnull().sum(axis=1)
    data['is_null_20-30'] = data.iloc[:, 30:40].isnull().sum(axis=1)
    data['is_null_40-50'] = data.iloc[:, 40:50].isnull().sum(axis=1)
    data['is_null_50-60'] = data.iloc[:, 50:60].isnull().sum(axis=1)
    data['is_null_60-80'] = data.iloc[:, 60:80].isnull().sum(axis=1)
    data['is_null_80-100'] = data.iloc[:, 80:100].isnull().sum(axis=1)
    data['is_null_100-110'] = data.iloc[:, 100:110].isnull().sum(axis=1)
    data['is_null_110-'] = data.iloc[:, 110:].isnull().sum(axis=1)
    data['is_null'] = data.iloc[:, :].isnull().sum(axis=1)
    data['F2110_LAG1-F2120_LAG1'] = data['F2110_LAG1']-data['F2120_LAG1']
    data['F2110-F2120'] = data['F2110']-data['F2120']
    data['sum_f']=0
    for name in data.columns:
        if '_LAG1' in name:
            data[name[:5]+"_diff"] = data[name] - data[name[:5]]
# #     data['sum_1100']=0
#     data['sum_1220']=0
#     data['sum_1260']=0
#     data['sum_1400']=0
#     data['sum_1530']=0
#     data['sum_2120']=0
    data['sum_2400']=data['F2400']-data['F2410']
#     for name in data.columns:
#         if name[0]=="F" and "LAG" not in name and '_diff' not in name:
#             numb = int(name[1:])
#             if numb>=1100 and numb<=1210:
#                 data['sum_1100']+=data[name]
#             if numb>=1220 and numb<=1250:
#                 data['sum_1220']+=data[name]
#             if numb>=1260 and numb<=1370:
#                 data['sum_1260']+=data[name]
#             if numb>=1400 and numb<=1520:
#                 data['sum_1400']+=data[name]
#             if numb>=1530 and numb<=1700:
#                 data['sum_1530']+=data[name] 
    return data
date_drop = ['SIGN_DATE_date','DATEFIRSTREG_date','TAXREGPAY_REGDATE_date','TAXREG_REGDATE_date']
def make_null_features_1(data,list_):
    for name in list_:
#         data[name+"_is_null"] = data[name].isnull() + 0
#         m = data[name].mean()
        data[name] = data[name].fillna(0) 
    data['SIGN_DATE_date'] = pd.to_datetime(data['SIGN_DATE'], format='%d%b%Y:%H:%M:%S')
    
    data['SIGN_DATE_year'] = data['SIGN_DATE_date'].dt.year
    data['SIGN_DATE_month'] = data['SIGN_DATE_date'].dt.month
    data['SIGN_DATE'] = data['SIGN_DATE_date'].dt.day
#     date_drop += ['SIGN_DATE_date','DATEFIRSTREG_date','TAXREGPAY_REGDATE_date','TAXREG_REGDATE_date']
    l = ['DATEFIRSTREG','TAXREG_REGDATE','TAXREGPAY_REGDATE']
    for col in l:
#         data[col+"_is_null"] = data[col].isnull() + 0
        data[col+'_date'] = pd.to_datetime(data[col], format='%d%b%Y:%H:%M:%S',infer_datetime_format=True).fillna(pd.Timestamp('19000101'))
        data[col+'day'] = data[col+'_date'].dt.day
        data[col+'month'] = data[col+'_date'].dt.month
        data[col] = data[col+'_date'].dt.year
#     data['SIGN_DATE_date_n'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n1'] = data.groupby('id_client')['SIGN_DATE_date'].diff(1).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n-1'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n2'] = data.groupby('id_client')['SIGN_DATE_date'].diff(2).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n-2'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-2).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n3'] = data.groupby('id_client')['SIGN_DATE_date'].diff(3).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n-3'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-3).dt.days.fillna(0).astype(int)
    data["NOTADMITTEDNUMBEREVER-ADMITTEDNUMBER_95_EVER"] = data['NOTADMITTEDNUMBER_EVER'] - data['ADMITTEDNUMBER_95_EVER']
#     data['TAXREG_REGDATE_n'] = data.groupby('id_client')['TAXREG_REGDATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['TAXREGPAY_REGDATE_n'] = data.groupby('id_client')['TAXREGPAY_REGDATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['sign-first'] = (data['SIGN_DATE_date']-data['DATEFIRSTREG_date']).dt.days.astype(int)
    data['sign-taxreg'] = (data['SIGN_DATE_date']-data['TAXREG_REGDATE_date']).dt.days.astype(int)
    data['sign-taxregpay'] = (data['SIGN_DATE_date']-data['TAXREGPAY_REGDATE_date']).dt.days.astype(int)
    data['taxreg-first'] = (data['TAXREG_REGDATE_date']-data['DATEFIRSTREG_date']).dt.days.astype(int)
    data['taxregpay-first'] = (data['TAXREGPAY_REGDATE_date']-data['DATEFIRSTREG_date']).dt.days.astype(int)
    data['taxregpay-taxreg'] = (data['TAXREGPAY_REGDATE_date']-data['TAXREG_REGDATE_date']).dt.days.astype(int)
#     data['WORKERSRANGE'] = data['WORKERSRANGE'].replace('5000 � �����', '5000..5000')
    data = data.join(data['WORKERSRANGE_1'].str.split(' .. ', 1, expand=True).rename(columns={0:'WORKERSRANGE1', 1:'WORKERSRANGE2'}))
    data['WORKERSRANGE1'] = data['WORKERSRANGE1'].replace('5000 � �����', '0')
    data['WORKERSRANGE2'] = data['WORKERSRANGE2'].replace('5000 � �����', '0')
    data['WORKERSRANGE1'] = data['WORKERSRANGE1'].fillna(0).astype(int)
    data['WORKERSRANGE2'] = data['WORKERSRANGE2'].fillna(0).astype(int)
    data['OKVED_CODE'] = data['OKVED_CODE'].fillna('0').astype(str)# +'.0.0.0'
    data = data.join(data['OKVED_CODE'].str.split('.', expand=True).rename(columns={0:'OKVED_CODE1', 1:'OKVED_CODE2', 2:'OKVED_CODE3'}))
    data['OKVED_CODE'] = data['OKVED_CODE'].str.replace('.','')
    data['OKVED_CODE1'] = data['OKVED_CODE1'].replace('', '0')
    data['OKVED_CODE2'] = data['OKVED_CODE2'].replace('', '0')
    data['OKVED_CODE3'] = data['OKVED_CODE3'].replace('', '0')
    data['OKVED_CODE1'] = data['OKVED_CODE1'].fillna(0).astype(int)
    data['OKVED_CODE2'] = data['OKVED_CODE2'].fillna(0).astype(int)
    data['OKVED_CODE3'] = data['OKVED_CODE3'].fillna(0).astype(int)
    data['SUM_233_EVER/WORKERSRANGE1'] = data['SUM_233_EVER']/data['WORKERSRANGE1']
#     data["new_f1"] = data["WINNERNUMBER_95_EVER"]/data["SIGNEDNUMBER_95_EVER"]
#     data["new_f2"] = data["WINNERNUMBER_233_EVER"]/data["SIGNEDNUMBER_233_EVER"]
#     data["new_f3"] = data["WINNERNUMBER_95_YEAR"]/data["SIGNEDNUMBER_95_YEAR"]
    data['ADMITTEDNUMBER_233_EVER/WORKERSRANGE1'] = data['ADMITTEDNUMBER_233_EVER']/data['WORKERSRANGE1']
    data['SUM_233_YEAR/WORKERSRANGE1'] = data['SUM_233_YEAR']/data['WORKERSRANGE1']
    data['ADMITTEDNUMBER_233_YEAR/WORKERSRANGE1'] = data['ADMITTEDNUMBER_233_YEAR']/data['WORKERSRANGE1']
    data['WINNERNUMBER_233_EVER/WORKERSRANGE1'] = data['WINNERNUMBER_233_EVER']/data['WORKERSRANGE1']
    data['NOTADMITTEDNUMBER_233_EVER/WORKERSRANGE1'] = data['NOTADMITTEDNUMBER_233_EVER']/data['WORKERSRANGE1']
    #     data['EGRPOINCLUDED/WORKERSRANGE1'] = data['EGRPOINCLUDED']/data['WORKERSRANGE1']
#     data['DATEFIRSTREG*WORKERSRANGE1'] = data['DATEFIRSTREG']*data['WORKERSRANGE1']
#     # sign-first
#     data['sign-first*WORKERSRANGE1'] = data['sign-first']*data['WORKERSRANGE1']
#     # TAXREG_REGDATE
#     data['TAXREG_REGDATE*WORKERSRANGE1'] = data['TAXREG_REGDATE']*data['WORKERSRANGE1']

    for name in ['OKFS_GROUP','OKOPF_GROUP','OKOGU_GROUP','WORKERSRANGE','SEX_NAME','CITIZENSHIP_NAME']:
        data[name] =data[name].astype('str').apply(lambda x: len(x))
        data[name] = data[name].fillna(0) 
    return data
data_train_1 = obr(data_train)
data_test_1 = obr(data_test)
data_train_1 = make_null_features(data_train)
data_test_1 = make_null_features(data_test)
with_na = desc.iloc[0,][desc.iloc[0,]!=17891].index.tolist()
with_na = ['FLAG_DISQUALIFICATION']
data_train_2 = make_null_features_1(data_train_1,with_na)
data_test_2 = make_null_features_1(data_test_1,with_na)
# data_train_3 = data_train_2.merge(make_cat_features(data_train_2,category_f), how='left', on='id_contract')
# data_test_3 = data_test_2.merge(make_cat_features(data_test_2,category_f), how='left', on='id_contract')

train_ = np.float32(data_train_2.drop([ "TARGET","BIRTHDATE",'WORKERSRANGE_1','id_contract','id_client']+date_drop, axis=1))
y_ = np.int64(data_train_2["TARGET"])
test_ = np.float32(data_test_2.drop(["BIRTHDATE",'WORKERSRANGE_1','id_contract','id_client']+date_drop, axis=1))

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:107: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


In [14]:
from catboost import CatBoostRegressor
catboost_1=CatBoostRegressor(n_estimators=1200,max_depth=11)
catboost_1.fit(train_,y_)

Learning rate set to 0.055683
0:	learn: 0.4779185	total: 734ms	remaining: 14m 40s
1:	learn: 0.4642169	total: 1.42s	remaining: 14m 13s
2:	learn: 0.4521175	total: 2.11s	remaining: 14m
3:	learn: 0.4400522	total: 2.81s	remaining: 13m 59s
4:	learn: 0.4291395	total: 3.51s	remaining: 13m 58s
5:	learn: 0.4190286	total: 4.18s	remaining: 13m 51s
6:	learn: 0.4092687	total: 4.88s	remaining: 13m 50s
7:	learn: 0.4006195	total: 5.59s	remaining: 13m 52s
8:	learn: 0.3926005	total: 6.27s	remaining: 13m 49s
9:	learn: 0.3852939	total: 6.97s	remaining: 13m 49s
10:	learn: 0.3780054	total: 7.69s	remaining: 13m 51s
11:	learn: 0.3714622	total: 8.56s	remaining: 14m 7s
12:	learn: 0.3656791	total: 9.52s	remaining: 14m 29s
13:	learn: 0.3602008	total: 10.5s	remaining: 14m 50s
14:	learn: 0.3552206	total: 11.3s	remaining: 14m 50s
15:	learn: 0.3503852	total: 12s	remaining: 14m 48s
16:	learn: 0.3457442	total: 12.9s	remaining: 14m 57s
17:	learn: 0.3415587	total: 13.7s	remaining: 14m 59s
18:	learn: 0.3373330	total: 14.5s

154:	learn: 0.2254828	total: 1m 55s	remaining: 12m 57s
155:	learn: 0.2247851	total: 1m 56s	remaining: 12m 56s
156:	learn: 0.2245167	total: 1m 56s	remaining: 12m 55s
157:	learn: 0.2239896	total: 1m 57s	remaining: 12m 54s
158:	learn: 0.2236799	total: 1m 58s	remaining: 12m 53s
159:	learn: 0.2233950	total: 1m 58s	remaining: 12m 51s
160:	learn: 0.2230687	total: 1m 59s	remaining: 12m 50s
161:	learn: 0.2227416	total: 2m	remaining: 12m 49s
162:	learn: 0.2222866	total: 2m	remaining: 12m 47s
163:	learn: 0.2220221	total: 2m 1s	remaining: 12m 45s
164:	learn: 0.2215327	total: 2m 1s	remaining: 12m 43s
165:	learn: 0.2211204	total: 2m 2s	remaining: 12m 42s
166:	learn: 0.2205929	total: 2m 3s	remaining: 12m 42s
167:	learn: 0.2203687	total: 2m 4s	remaining: 12m 41s
168:	learn: 0.2201674	total: 2m 4s	remaining: 12m 41s
169:	learn: 0.2200037	total: 2m 5s	remaining: 12m 41s
170:	learn: 0.2195712	total: 2m 6s	remaining: 12m 41s
171:	learn: 0.2192211	total: 2m 7s	remaining: 12m 40s
172:	learn: 0.2186455	total

305:	learn: 0.1812186	total: 3m 41s	remaining: 10m 48s
306:	learn: 0.1809436	total: 3m 42s	remaining: 10m 47s
307:	learn: 0.1808100	total: 3m 43s	remaining: 10m 46s
308:	learn: 0.1806360	total: 3m 44s	remaining: 10m 46s
309:	learn: 0.1802681	total: 3m 44s	remaining: 10m 45s
310:	learn: 0.1802507	total: 3m 45s	remaining: 10m 44s
311:	learn: 0.1799558	total: 3m 46s	remaining: 10m 43s
312:	learn: 0.1797869	total: 3m 46s	remaining: 10m 42s
313:	learn: 0.1796011	total: 3m 47s	remaining: 10m 41s
314:	learn: 0.1793856	total: 3m 48s	remaining: 10m 40s
315:	learn: 0.1792167	total: 3m 48s	remaining: 10m 40s
316:	learn: 0.1790114	total: 3m 49s	remaining: 10m 39s
317:	learn: 0.1788069	total: 3m 50s	remaining: 10m 38s
318:	learn: 0.1786166	total: 3m 50s	remaining: 10m 37s
319:	learn: 0.1783243	total: 3m 51s	remaining: 10m 36s
320:	learn: 0.1781711	total: 3m 52s	remaining: 10m 35s
321:	learn: 0.1779248	total: 3m 52s	remaining: 10m 34s
322:	learn: 0.1777544	total: 3m 53s	remaining: 10m 34s
323:	learn

457:	learn: 0.1537286	total: 5m 26s	remaining: 8m 49s
458:	learn: 0.1535873	total: 5m 27s	remaining: 8m 49s
459:	learn: 0.1533457	total: 5m 28s	remaining: 8m 48s
460:	learn: 0.1531722	total: 5m 29s	remaining: 8m 47s
461:	learn: 0.1530122	total: 5m 29s	remaining: 8m 46s
462:	learn: 0.1529089	total: 5m 30s	remaining: 8m 45s
463:	learn: 0.1527282	total: 5m 31s	remaining: 8m 45s
464:	learn: 0.1525279	total: 5m 31s	remaining: 8m 44s
465:	learn: 0.1523148	total: 5m 32s	remaining: 8m 43s
466:	learn: 0.1521397	total: 5m 33s	remaining: 8m 42s
467:	learn: 0.1520020	total: 5m 33s	remaining: 8m 42s
468:	learn: 0.1519594	total: 5m 34s	remaining: 8m 41s
469:	learn: 0.1518447	total: 5m 35s	remaining: 8m 40s
470:	learn: 0.1515413	total: 5m 35s	remaining: 8m 39s
471:	learn: 0.1513580	total: 5m 36s	remaining: 8m 39s
472:	learn: 0.1509816	total: 5m 37s	remaining: 8m 38s
473:	learn: 0.1507752	total: 5m 37s	remaining: 8m 37s
474:	learn: 0.1506394	total: 5m 38s	remaining: 8m 36s
475:	learn: 0.1505779	total:

610:	learn: 0.1312471	total: 7m 19s	remaining: 7m 3s
611:	learn: 0.1310152	total: 7m 20s	remaining: 7m 3s
612:	learn: 0.1308516	total: 7m 21s	remaining: 7m 2s
613:	learn: 0.1307162	total: 7m 22s	remaining: 7m 1s
614:	learn: 0.1305740	total: 7m 22s	remaining: 7m 1s
615:	learn: 0.1304861	total: 7m 23s	remaining: 7m
616:	learn: 0.1304215	total: 7m 24s	remaining: 7m
617:	learn: 0.1302953	total: 7m 25s	remaining: 6m 59s
618:	learn: 0.1302082	total: 7m 26s	remaining: 6m 58s
619:	learn: 0.1300330	total: 7m 27s	remaining: 6m 58s
620:	learn: 0.1299087	total: 7m 28s	remaining: 6m 57s
621:	learn: 0.1297293	total: 7m 28s	remaining: 6m 57s
622:	learn: 0.1296224	total: 7m 29s	remaining: 6m 56s
623:	learn: 0.1295950	total: 7m 30s	remaining: 6m 55s
624:	learn: 0.1293932	total: 7m 30s	remaining: 6m 54s
625:	learn: 0.1292683	total: 7m 31s	remaining: 6m 53s
626:	learn: 0.1291439	total: 7m 32s	remaining: 6m 53s
627:	learn: 0.1289837	total: 7m 32s	remaining: 6m 52s
628:	learn: 0.1288626	total: 7m 33s	remai

763:	learn: 0.1137972	total: 9m 11s	remaining: 5m 14s
764:	learn: 0.1136252	total: 9m 11s	remaining: 5m 13s
765:	learn: 0.1135868	total: 9m 12s	remaining: 5m 12s
766:	learn: 0.1134929	total: 9m 13s	remaining: 5m 12s
767:	learn: 0.1133802	total: 9m 13s	remaining: 5m 11s
768:	learn: 0.1133299	total: 9m 14s	remaining: 5m 10s
769:	learn: 0.1132805	total: 9m 15s	remaining: 5m 10s
770:	learn: 0.1131173	total: 9m 15s	remaining: 5m 9s
771:	learn: 0.1129658	total: 9m 16s	remaining: 5m 8s
772:	learn: 0.1128991	total: 9m 17s	remaining: 5m 7s
773:	learn: 0.1128044	total: 9m 18s	remaining: 5m 7s
774:	learn: 0.1127537	total: 9m 18s	remaining: 5m 6s
775:	learn: 0.1126540	total: 9m 19s	remaining: 5m 5s
776:	learn: 0.1125281	total: 9m 20s	remaining: 5m 4s
777:	learn: 0.1124840	total: 9m 20s	remaining: 5m 4s
778:	learn: 0.1123709	total: 9m 21s	remaining: 5m 3s
779:	learn: 0.1122656	total: 9m 22s	remaining: 5m 2s
780:	learn: 0.1121280	total: 9m 22s	remaining: 5m 1s
781:	learn: 0.1120078	total: 9m 23s	rem

915:	learn: 0.1001836	total: 11m 6s	remaining: 3m 26s
916:	learn: 0.1001392	total: 11m 7s	remaining: 3m 25s
917:	learn: 0.1001353	total: 11m 8s	remaining: 3m 25s
918:	learn: 0.1001025	total: 11m 8s	remaining: 3m 24s
919:	learn: 0.1000479	total: 11m 9s	remaining: 3m 23s
920:	learn: 0.0999685	total: 11m 10s	remaining: 3m 23s
921:	learn: 0.0998469	total: 11m 10s	remaining: 3m 22s
922:	learn: 0.0997838	total: 11m 11s	remaining: 3m 21s
923:	learn: 0.0996699	total: 11m 12s	remaining: 3m 20s
924:	learn: 0.0996631	total: 11m 12s	remaining: 3m 20s
925:	learn: 0.0995839	total: 11m 13s	remaining: 3m 19s
926:	learn: 0.0995402	total: 11m 14s	remaining: 3m 18s
927:	learn: 0.0995197	total: 11m 15s	remaining: 3m 17s
928:	learn: 0.0994388	total: 11m 16s	remaining: 3m 17s
929:	learn: 0.0994217	total: 11m 16s	remaining: 3m 16s
930:	learn: 0.0993708	total: 11m 17s	remaining: 3m 15s
931:	learn: 0.0992410	total: 11m 18s	remaining: 3m 15s
932:	learn: 0.0991694	total: 11m 18s	remaining: 3m 14s
933:	learn: 0.0

1064:	learn: 0.0907205	total: 12m 55s	remaining: 1m 38s
1065:	learn: 0.0905900	total: 12m 56s	remaining: 1m 37s
1066:	learn: 0.0905374	total: 12m 56s	remaining: 1m 36s
1067:	learn: 0.0905048	total: 12m 57s	remaining: 1m 36s
1068:	learn: 0.0904517	total: 12m 58s	remaining: 1m 35s
1069:	learn: 0.0904022	total: 12m 58s	remaining: 1m 34s
1070:	learn: 0.0903708	total: 12m 59s	remaining: 1m 33s
1071:	learn: 0.0902952	total: 13m	remaining: 1m 33s
1072:	learn: 0.0902634	total: 13m	remaining: 1m 32s
1073:	learn: 0.0902291	total: 13m 1s	remaining: 1m 31s
1074:	learn: 0.0902018	total: 13m 2s	remaining: 1m 30s
1075:	learn: 0.0901427	total: 13m 2s	remaining: 1m 30s
1076:	learn: 0.0901214	total: 13m 3s	remaining: 1m 29s
1077:	learn: 0.0900664	total: 13m 4s	remaining: 1m 28s
1078:	learn: 0.0900105	total: 13m 4s	remaining: 1m 28s
1079:	learn: 0.0899303	total: 13m 5s	remaining: 1m 27s
1080:	learn: 0.0898735	total: 13m 6s	remaining: 1m 26s
1081:	learn: 0.0897258	total: 13m 6s	remaining: 1m 25s
1082:	lea

In [15]:
a = catboost_1.predict(test_)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_3.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
catboost_1.save_model("model_2_.cbm")
# catboost_1=CatBoostRegressor(n_estimators=1200,max_depth=11)
# catboost_1.load_model("model_1_.cbm")

In [17]:
import catboost
imp = catboost_1.get_feature_importance(catboost.Pool(train_, y_))
feat = list(data_train_2.drop([ "TARGET","BIRTHDATE",'WORKERSRANGE_1','id_contract','id_client']+date_drop, axis=1).columns)
sort = [(imp[i],feat[i]) for i in range(len(imp))]
sorted_by_0 = sorted(sort, key=lambda tup: tup[0], reverse=True)
bad_f = []
for i in range(len(imp)):
    print(sorted_by_0[i][1], round(sorted_by_0[i][0], 2))

SIGN_DATE_date_n3 4.87
SIGN_DATE_date_n-3 4.83
OKVED_CODE2 4.29
OKVED_CODE1 4.22
sign-first 3.68
OKVED_CODE 3.27
WORKERSRANGE 2.54
OKTMO_CODE 2.43
NOTADMITTEDNUMBEREVER-ADMITTEDNUMBER_95_EVER 2.39
DATEFIRSTREG 2.08
SIGN_DATE_month 1.92
sign-taxregpay 1.62
TAXREG_REGDATE 1.61
OKATO_FED 1.6
DATEFIRSTREGday 1.55
sign-taxreg 1.53
WORKERSRANGE2 1.44
TELEPHONECOUNT 1.42
TAXREG_REGDATEday 1.41
OKTMO_FED 1.37
SIGN_DATE_date_n2 1.36
TAXREG_REGDATEmonth 1.33
SIGN_DATE 1.33
SIGN_DATE_date_n1 1.26
DATEFIRSTREGmonth 1.25
OKATO_REGIONCODE 1.24
F2120 1.15
SIGN_DATE_date_n-2 1.12
SIGN_DATE_date_n-1 1.04
SIGNEDNUMBER_95_EVER 0.93
DEFENDANT_SUM_EVER 0.93
is_null 0.9
F1250 0.85
F2110_diff 0.81
taxregpay-first 0.8
WORKERSRANGE1 0.76
F2110 0.73
AGE 0.71
F1230 0.71
SUM_95_EVER 0.67
F1520 0.66
WINNERNUMBER_95_EVER 0.63
F2200 0.6
is_null_110- 0.59
F2120_diff 0.59
F1510 0.59
ADMITTEDNUMBER_233_EVER/WORKERSRANGE1 0.58
MANAGERCOUNTINREGION 0.54
F2300_diff 0.53
taxregpay-taxreg 0.53
MANAGERCOUNTINCOUNTRY 0.53
F23

In [18]:
ind_mean=list(data_train_2.groupby("id_client").mean()['TARGET'].index)
mean_=data_train_2.groupby("id_client").mean()['TARGET']
ind_count=list(data_train_2.groupby("id_client").count()['TARGET'].index)
count_=data_train_2.groupby("id_client").count()['TARGET']
ind_sum=list(data_train_2.groupby("id_client").sum()['TARGET'].index)
sum_=data_train_2.groupby("id_client").sum()['TARGET']
list_cl_tr = list(data_train_2['id_client'])
list_cl_t = list(data_test_2['id_client'])
list_cl_tr_m = []
list_cl_tr_c = []
list_cl_tr_s = []
list_cl_t_m = []
list_cl_t_c = []
list_cl_t_s = []
for i in list_cl_tr:
#     print(i)
    if count_[i] ==1:
        list_cl_tr_m.append(0)
        list_cl_tr_c.append(0)
        list_cl_tr_s.append(0)
    else:
        list_cl_tr_m.append(mean_[i])
        list_cl_tr_c.append(count_[i])
        list_cl_tr_s.append(sum_[i])
for i in list_cl_t:
#     print(i)
    if i in list_cl_tr:
        list_cl_t_m.append(mean_[i])
        list_cl_t_c.append(count_[i])
        list_cl_t_s.append(sum_[i])
    else:
        list_cl_t_m.append(-1)
        list_cl_t_c.append(0)
        list_cl_t_s.append(0)
data_train_2['mean'] = list_cl_tr_m
data_test_2['mean'] = list_cl_t_m
data_train_2['count'] = list_cl_tr_c
data_test_2['count'] = list_cl_t_c
data_train_2['sum'] = list_cl_tr_s
data_test_2['sum'] = list_cl_t_s

In [19]:
pred_stat1= list(pd.read_csv('./subm_3.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat2= list_cl_t_m
res = []
c = 0
for i in range(len(pred_stat1)):
    if pred_stat2[i] ==-1:
        res.append(pred_stat1[i])
    else:
        c+=1
#         print(pred_stat2[i],pred_stat1[i])
        if pred_stat2[i]>0:
            res.append(pred_stat1[i]+0.07*pred_stat2[i])
        if pred_stat2[i]==0:
            res.append(pred_stat1[i]-0.07)
a = np.array(res)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_4.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [37]:
from sklearn.base import BaseEstimator, ClassifierMixin
import lightgbm as lgb
import xgboost as xgb
# model1 = lgb.LGBMClassifier(num_leaves=5, learning_rate=0.05, n_estimators=int(600*2),
#                                           colsample_bytree=0.5, subsample=0.5,
#                                           nthread=-1, random_state=3)
model2 = lgb.LGBMClassifier(num_leaves=40,
                               learning_rate=0.05,
                               n_estimators=400)

In [38]:
model2.fit(train_,y_)

LGBMClassifier(learning_rate=0.05, n_estimators=400, num_leaves=40)

In [40]:
a = model2.predict_proba(test_)[:, 1]
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_9.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [42]:
ind_mean=list(data_train_2.groupby("id_client").mean()['TARGET'].index)
mean_=data_train_2.groupby("id_client").mean()['TARGET']
ind_count=list(data_train_2.groupby("id_client").count()['TARGET'].index)
count_=data_train_2.groupby("id_client").count()['TARGET']
ind_sum=list(data_train_2.groupby("id_client").sum()['TARGET'].index)
sum_=data_train_2.groupby("id_client").sum()['TARGET']
list_cl_tr = list(data_train_2['id_client'])
list_cl_t = list(data_test_2['id_client'])
list_cl_tr_m = []
list_cl_tr_c = []
list_cl_tr_s = []
list_cl_t_m = []
list_cl_t_c = []
list_cl_t_s = []
for i in list_cl_tr:
#     print(i)
    if count_[i] ==1:
        list_cl_tr_m.append(0)
        list_cl_tr_c.append(0)
        list_cl_tr_s.append(0)
    else:
        list_cl_tr_m.append(mean_[i])
        list_cl_tr_c.append(count_[i])
        list_cl_tr_s.append(sum_[i])
for i in list_cl_t:
#     print(i)
    if i in list_cl_tr:
        list_cl_t_m.append(mean_[i])
        list_cl_t_c.append(count_[i])
        list_cl_t_s.append(sum_[i])
    else:
        list_cl_t_m.append(-1)
        list_cl_t_c.append(0)
        list_cl_t_s.append(0)
data_train_2['mean'] = list_cl_tr_m
data_test_2['mean'] = list_cl_t_m
data_train_2['count'] = list_cl_tr_c
data_test_2['count'] = list_cl_t_c
data_train_2['sum'] = list_cl_tr_s
data_test_2['sum'] = list_cl_t_s

In [43]:
pred_stat1= list(pd.read_csv('./subm_9.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat2= list_cl_t_m
res = []
c = 0
for i in range(len(pred_stat1)):
    if pred_stat2[i] ==-1:
        res.append(pred_stat1[i])
    else:
        c+=1
#         print(pred_stat2[i],pred_stat1[i])
        if pred_stat2[i]>0:
            res.append(pred_stat1[i]+0.07*pred_stat2[i])
        if pred_stat2[i]==0:
            res.append(pred_stat1[i]-0.07)
a = np.array(res)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_10.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
def obr(df):
#     df=df.drop(["id_contract","id_client","BIRTHDATE"], axis=1)
    columns_sum_LAG = [i for i in data_train.columns if i[0]=="F" and len(i)>=5]
    columns_sum = [i for i in data_train.columns if i[0]=="F" and len(i)==5]
    for i in columns_sum:
        df[i].fillna(0, inplace=True)
        df['SUM_F'] = data_train.loc[:, "F1100":"F2410"].sum(axis=1)
        df['SUM_F_LAGS'] = data_train.loc[:, "F1150_LAG1":"F2400_LAG1"].sum(axis=1)
    return df
data_train = pd.read_csv('train_dataset_hackathon_mkb.csv',sep = ';')
desc = data_train.describe()
data_test = pd.read_csv('test_dataset_hackathon_mkb.csv',sep = ';')
data_train['WORKERSRANGE_1'] = data_train['WORKERSRANGE']
data_test['WORKERSRANGE_1'] = data_test['WORKERSRANGE']
category_f = ['OKFS_GROUP','OKOPF_GROUP',
                    'OKATO_FED','EGRPOINCLUDED','WORKERSRANGE_1']


def make_null_features(data):
     
    data['is_null_0-10'] = data.iloc[:, 0:10].isnull().sum(axis=1)
    data['is_null_10-20'] = data.iloc[:, 10:20].isnull().sum(axis=1)
    data['is_null_20-30'] = data.iloc[:, 20:30].isnull().sum(axis=1)
    data['is_null_20-30'] = data.iloc[:, 30:40].isnull().sum(axis=1)
    data['is_null_40-50'] = data.iloc[:, 40:50].isnull().sum(axis=1)
    data['is_null_50-60'] = data.iloc[:, 50:60].isnull().sum(axis=1)
    data['is_null_60-80'] = data.iloc[:, 60:80].isnull().sum(axis=1)
    data['is_null_80-100'] = data.iloc[:, 80:100].isnull().sum(axis=1)
    data['is_null_100-110'] = data.iloc[:, 100:110].isnull().sum(axis=1)
    data['is_null_110-'] = data.iloc[:, 110:].isnull().sum(axis=1)
    data['is_null'] = data.iloc[:, :].isnull().sum(axis=1)
    data['F2110_LAG1-F2120_LAG1'] = data['F2110_LAG1']-data['F2120_LAG1']
    data['F2110-F2120'] = data['F2110']-data['F2120']
    data['sum_f']=0
    for name in data.columns:
        if '_LAG1' in name:
            data[name[:5]+"_diff"] = data[name] - data[name[:5]]
# #     data['sum_1100']=0
#     data['sum_1220']=0
#     data['sum_1260']=0
#     data['sum_1400']=0
#     data['sum_1530']=0
#     data['sum_2120']=0
    data['sum_2400']=data['F2400']-data['F2410']
#     for name in data.columns:
#         if name[0]=="F" and "LAG" not in name and '_diff' not in name:
#             numb = int(name[1:])
#             if numb>=1100 and numb<=1210:
#                 data['sum_1100']+=data[name]
#             if numb>=1220 and numb<=1250:
#                 data['sum_1220']+=data[name]
#             if numb>=1260 and numb<=1370:
#                 data['sum_1260']+=data[name]
#             if numb>=1400 and numb<=1520:
#                 data['sum_1400']+=data[name]
#             if numb>=1530 and numb<=1700:
#                 data['sum_1530']+=data[name] 
    return data
date_drop = ['SIGN_DATE_date','DATEFIRSTREG_date','TAXREGPAY_REGDATE_date','TAXREG_REGDATE_date']
def make_null_features_1(data,list_):
    for name in list_:
#         data[name+"_is_null"] = data[name].isnull() + 0
#         m = data[name].mean()
        data[name] = data[name].fillna(0) 
    data['SIGN_DATE_date'] = pd.to_datetime(data['SIGN_DATE'], format='%d%b%Y:%H:%M:%S')
    
    data['SIGN_DATE_year'] = data['SIGN_DATE_date'].dt.year
    data['SIGN_DATE_month'] = data['SIGN_DATE_date'].dt.month
    data['SIGN_DATE'] = data['SIGN_DATE_date'].dt.day
#     date_drop += ['SIGN_DATE_date','DATEFIRSTREG_date','TAXREGPAY_REGDATE_date','TAXREG_REGDATE_date']
    l = ['DATEFIRSTREG','TAXREG_REGDATE','TAXREGPAY_REGDATE']
    for col in l:
#         data[col+"_is_null"] = data[col].isnull() + 0
        data[col+'_date'] = pd.to_datetime(data[col], format='%d%b%Y:%H:%M:%S',infer_datetime_format=True).fillna(pd.Timestamp('19000101'))
        data[col+'day'] = data[col+'_date'].dt.day
        data[col+'month'] = data[col+'_date'].dt.month
        data[col] = data[col+'_date'].dt.year
#     data['SIGN_DATE_date_n'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n1'] = data.groupby('id_client')['SIGN_DATE_date'].diff(1).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n-1'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n2'] = data.groupby('id_client')['SIGN_DATE_date'].diff(2).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n-2'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-2).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n3'] = data.groupby('id_client')['SIGN_DATE_date'].diff(3).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_n-3'] = data.groupby('id_client')['SIGN_DATE_date'].diff(-3).dt.days.fillna(0).astype(int)
    data['SIGN_DATE_date_1_mean'] = data.loc[:, ['SIGN_DATE_date_n1','SIGN_DATE_date_n2','SIGN_DATE_date_n3']].mean(axis=1)
    data['SIGN_DATE_date_-1_mean'] = data.loc[:, ['SIGN_DATE_date_n-1','SIGN_DATE_date_n-2','SIGN_DATE_date_n-3']].mean(axis=1)
    
    data["NOTADMITTEDNUMBEREVER-ADMITTEDNUMBER_95_EVER"] = data['NOTADMITTEDNUMBER_EVER'] - data['ADMITTEDNUMBER_95_EVER']
#     data['TAXREG_REGDATE_n'] = data.groupby('id_client')['TAXREG_REGDATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['TAXREGPAY_REGDATE_n'] = data.groupby('id_client')['TAXREGPAY_REGDATE_date'].diff(-1).dt.days.fillna(0).astype(int)
    data['sign-first'] = (data['SIGN_DATE_date']-data['DATEFIRSTREG_date']).dt.days.astype(int)
    data['sign-taxreg'] = (data['SIGN_DATE_date']-data['TAXREG_REGDATE_date']).dt.days.astype(int)
    data['sign-taxregpay'] = (data['SIGN_DATE_date']-data['TAXREGPAY_REGDATE_date']).dt.days.astype(int)
    data['taxreg-first'] = (data['TAXREG_REGDATE_date']-data['DATEFIRSTREG_date']).dt.days.astype(int)
    data['taxregpay-first'] = (data['TAXREGPAY_REGDATE_date']-data['DATEFIRSTREG_date']).dt.days.astype(int)
    data['taxregpay-taxreg'] = (data['TAXREGPAY_REGDATE_date']-data['TAXREG_REGDATE_date']).dt.days.astype(int)
#     data['WORKERSRANGE'] = data['WORKERSRANGE'].replace('5000 � �����', '5000..5000')
    data = data.join(data['WORKERSRANGE_1'].str.split(' .. ', 1, expand=True).rename(columns={0:'WORKERSRANGE1', 1:'WORKERSRANGE2'}))
    data['WORKERSRANGE1'] = data['WORKERSRANGE1'].replace('5000 � �����', '0')
    data['WORKERSRANGE2'] = data['WORKERSRANGE2'].replace('5000 � �����', '0')
    data['WORKERSRANGE1'] = data['WORKERSRANGE1'].fillna(0).astype(int)
    data['WORKERSRANGE2'] = data['WORKERSRANGE2'].fillna(0).astype(int)
    data['OKVED_CODE'] = data['OKVED_CODE'].fillna('0').astype(str)# +'.0.0.0'
    data = data.join(data['OKVED_CODE'].str.split('.', expand=True).rename(columns={0:'OKVED_CODE1', 1:'OKVED_CODE2', 2:'OKVED_CODE3'}))
    data['OKVED_CODE'] = data['OKVED_CODE'].str.replace('.','')
    data['OKVED_CODE1'] = data['OKVED_CODE1'].replace('', '0')
    data['OKVED_CODE2'] = data['OKVED_CODE2'].replace('', '0')
    data['OKVED_CODE3'] = data['OKVED_CODE3'].replace('', '0')
    data['OKVED_CODE1'] = data['OKVED_CODE1'].fillna(0).astype(int)
    data['OKVED_CODE2'] = data['OKVED_CODE2'].fillna(0).astype(int)
    data['OKVED_CODE3'] = data['OKVED_CODE3'].fillna(0).astype(int)
    data['SUM_233_EVER/WORKERSRANGE1'] = data['SUM_233_EVER']/data['WORKERSRANGE1']
    data['ADMITTEDNUMBER_233_EVER/WORKERSRANGE1'] = data['ADMITTEDNUMBER_233_EVER']/data['WORKERSRANGE1']
    data['SUM_233_YEAR/WORKERSRANGE1'] = data['SUM_233_YEAR']/data['WORKERSRANGE1']
    data['ADMITTEDNUMBER_233_YEAR/WORKERSRANGE1'] = data['ADMITTEDNUMBER_233_YEAR']/data['WORKERSRANGE1']
    data['WINNERNUMBER_233_EVER/WORKERSRANGE1'] = data['WINNERNUMBER_233_EVER']/data['WORKERSRANGE1']
    data['NOTADMITTEDNUMBER_233_EVER/WORKERSRANGE1'] = data['NOTADMITTEDNUMBER_233_EVER']/data['WORKERSRANGE1']
#     data['WORKERSRANGE2_n1'] = data.groupby('id_client')['WORKERSRANGE2'].diff(1).astype(int)
#     data['WORKERSRANGE2_n-1'] = data.groupby('id_client')['WORKERSRANGE2'].diff(-1).astype(int)
#     data['F2110_n1'] = data.groupby('id_client')['F2110'].diff(1).astype(int)
#     data['F2110_n-1'] = data.groupby('id_client')['F2110'].diff(-1).astype(int)
#     data['IP_n1'] = data.groupby('id_client')['IP'].diff(1).astype(int)
#     data['IP_n-1'] = data.groupby('id_client')['IP'].diff(-1).astype(int)
#     data['SUM_95_YEAR_n1'] = data.groupby('id_client')['SUM_95_YEAR'].diff(1).astype(int)
#     data['SUM_95_YEAR_n-1'] = data.groupby('id_client')['SUM_95_YEAR'].diff(-1).astype(int)
#     data['WORKERSRANGE2_n1'] = data.groupby('id_client')['WORKERSRANGE2'].diff(1).astype(int)
#     data['WORKERSRANGE2_n-1'] = data.groupby('id_client')['WORKERSRANGE2'].diff(-1).astype(int)
#     data['F2110_n1'] = data.groupby('id_client')['F2110'].diff(1)
#     data['F2110_n-1'] = data.groupby('id_client')['F2110'].diff(-1)
    data['IP_n1'] = data.groupby('id_client')['IP_flag'].diff(1)
    data['IP_n-1'] = data.groupby('id_client')['IP_flag'].diff(-1)
    data['SUM_95_YEAR_n1'] = data.groupby('id_client')['SUM_95_YEAR'].diff(1)
    data['SUM_95_YEAR_n-1'] = data.groupby('id_client')['SUM_95_YEAR'].diff(-1)
    data['is_null_n1'] = data.groupby('id_client')['is_null'].diff(1)
    data['is_null_n-1'] = data.groupby('id_client')['is_null'].diff(-1)
    #     data['EGRPOINCLUDED/WORKERSRANGE1'] = data['EGRPOINCLUDED']/data['WORKERSRANGE1']
#     data['DATEFIRSTREG*WORKERSRANGE1'] = data['DATEFIRSTREG']*data['WORKERSRANGE1']
#     # sign-first
#     data['sign-first*WORKERSRANGE1'] = data['sign-first']*data['WORKERSRANGE1']
#     # TAXREG_REGDATE
#     data['TAXREG_REGDATE*WORKERSRANGE1'] = data['TAXREG_REGDATE']*data['WORKERSRANGE1']

    for name in ['OKFS_GROUP','OKOPF_GROUP','OKOGU_GROUP','WORKERSRANGE','SEX_NAME','CITIZENSHIP_NAME']:
        data[name] =data[name].astype('str').apply(lambda x: len(x))
        data[name] = data[name].fillna(0) 
#     data = data.sort_values(by='SIGN_DATE_date')
    return data
data_train_1 = obr(data_train)
data_test_1 = obr(data_test)
data_train_1 = make_null_features(data_train_1)
data_test_1 = make_null_features(data_test_1)
with_na = desc.iloc[0,][desc.iloc[0,]!=17891].index.tolist()
with_na = ['FLAG_DISQUALIFICATION']
data_train_2 = make_null_features_1(data_train_1,with_na)
data_test_2 = make_null_features_1(data_test_1,with_na)
# data_train_3 = data_train_2.merge(make_cat_features(data_train_2,category_f), how='left', on='id_contract')
# data_test_3 = data_test_2.merge(make_cat_features(data_test_2,category_f), how='left', on='id_contract')
# data_train_2 = data_train_2.replace([np.inf, -np.inf], 0, inplace=True)
# data_test_2 = data_test_2.replace([np.inf, -np.inf], 0, inplace=True)
train_ = np.float32(data_train_2.drop([ "TARGET","BIRTHDATE",'WORKERSRANGE_1','id_contract']+date_drop, axis=1))
y_ = np.int64(data_train_2["TARGET"])
test_ = np.float32(data_test_2.drop(["BIRTHDATE",'WORKERSRANGE_1','id_contract']+date_drop, axis=1))

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:110: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


In [21]:
from catboost import CatBoostRegressor
catboost_1=CatBoostRegressor(n_estimators=1200,max_depth=11)
catboost_1.fit(train_,y_)

Learning rate set to 0.055683
0:	learn: 0.4771264	total: 968ms	remaining: 19m 20s
1:	learn: 0.4627739	total: 1.72s	remaining: 17m 11s
2:	learn: 0.4491607	total: 2.42s	remaining: 16m 7s
3:	learn: 0.4368898	total: 3.15s	remaining: 15m 41s
4:	learn: 0.4251607	total: 3.88s	remaining: 15m 27s
5:	learn: 0.4139069	total: 4.63s	remaining: 15m 22s
6:	learn: 0.4034433	total: 5.39s	remaining: 15m 19s
7:	learn: 0.3938479	total: 6.12s	remaining: 15m 11s
8:	learn: 0.3852696	total: 6.84s	remaining: 15m 4s
9:	learn: 0.3779797	total: 7.67s	remaining: 15m 12s
10:	learn: 0.3703757	total: 8.6s	remaining: 15m 29s
11:	learn: 0.3628680	total: 9.44s	remaining: 15m 34s
12:	learn: 0.3560430	total: 10.3s	remaining: 15m 40s
13:	learn: 0.3497785	total: 11.2s	remaining: 15m 50s
14:	learn: 0.3439127	total: 12s	remaining: 15m 51s
15:	learn: 0.3382832	total: 12.8s	remaining: 15m 47s
16:	learn: 0.3335959	total: 13.5s	remaining: 15m 41s
17:	learn: 0.3293168	total: 14.2s	remaining: 15m 34s
18:	learn: 0.3251108	total: 14.

154:	learn: 0.2199972	total: 1m 54s	remaining: 12m 52s
155:	learn: 0.2197685	total: 1m 55s	remaining: 12m 51s
156:	learn: 0.2195586	total: 1m 55s	remaining: 12m 50s
157:	learn: 0.2192063	total: 1m 56s	remaining: 12m 49s
158:	learn: 0.2187742	total: 1m 57s	remaining: 12m 48s
159:	learn: 0.2185029	total: 1m 58s	remaining: 12m 48s
160:	learn: 0.2180796	total: 1m 59s	remaining: 12m 48s
161:	learn: 0.2174721	total: 1m 59s	remaining: 12m 48s
162:	learn: 0.2169672	total: 2m	remaining: 12m 48s
163:	learn: 0.2167427	total: 2m 1s	remaining: 12m 48s
164:	learn: 0.2164627	total: 2m 2s	remaining: 12m 47s
165:	learn: 0.2161335	total: 2m 3s	remaining: 12m 46s
166:	learn: 0.2158992	total: 2m 3s	remaining: 12m 45s
167:	learn: 0.2153969	total: 2m 4s	remaining: 12m 45s
168:	learn: 0.2152971	total: 2m 5s	remaining: 12m 44s
169:	learn: 0.2148603	total: 2m 6s	remaining: 12m 43s
170:	learn: 0.2144142	total: 2m 6s	remaining: 12m 42s
171:	learn: 0.2140623	total: 2m 7s	remaining: 12m 42s
172:	learn: 0.2136307	t

304:	learn: 0.1790232	total: 3m 56s	remaining: 11m 32s
305:	learn: 0.1789945	total: 3m 56s	remaining: 11m 32s
306:	learn: 0.1789664	total: 3m 57s	remaining: 11m 31s
307:	learn: 0.1788205	total: 3m 59s	remaining: 11m 32s
308:	learn: 0.1785152	total: 4m	remaining: 11m 34s
309:	learn: 0.1782009	total: 4m 1s	remaining: 11m 34s
310:	learn: 0.1779933	total: 4m 2s	remaining: 11m 33s
311:	learn: 0.1777612	total: 4m 3s	remaining: 11m 32s
312:	learn: 0.1775106	total: 4m 3s	remaining: 11m 30s
313:	learn: 0.1773433	total: 4m 4s	remaining: 11m 28s
314:	learn: 0.1772772	total: 4m 4s	remaining: 11m 27s
315:	learn: 0.1770632	total: 4m 5s	remaining: 11m 25s
316:	learn: 0.1769544	total: 4m 5s	remaining: 11m 24s
317:	learn: 0.1767394	total: 4m 6s	remaining: 11m 22s
318:	learn: 0.1765625	total: 4m 6s	remaining: 11m 21s
319:	learn: 0.1763875	total: 4m 7s	remaining: 11m 19s
320:	learn: 0.1763812	total: 4m 7s	remaining: 11m 17s
321:	learn: 0.1761596	total: 4m 8s	remaining: 11m 16s
322:	learn: 0.1759716	total

456:	learn: 0.1534206	total: 5m 20s	remaining: 8m 41s
457:	learn: 0.1532732	total: 5m 21s	remaining: 8m 40s
458:	learn: 0.1531051	total: 5m 21s	remaining: 8m 39s
459:	learn: 0.1529443	total: 5m 22s	remaining: 8m 38s
460:	learn: 0.1528357	total: 5m 23s	remaining: 8m 37s
461:	learn: 0.1525513	total: 5m 23s	remaining: 8m 37s
462:	learn: 0.1523690	total: 5m 24s	remaining: 8m 36s
463:	learn: 0.1522975	total: 5m 24s	remaining: 8m 35s
464:	learn: 0.1522792	total: 5m 25s	remaining: 8m 34s
465:	learn: 0.1520951	total: 5m 26s	remaining: 8m 33s
466:	learn: 0.1518500	total: 5m 26s	remaining: 8m 32s
467:	learn: 0.1516861	total: 5m 27s	remaining: 8m 31s
468:	learn: 0.1516030	total: 5m 27s	remaining: 8m 31s
469:	learn: 0.1513402	total: 5m 28s	remaining: 8m 30s
470:	learn: 0.1512561	total: 5m 29s	remaining: 8m 29s
471:	learn: 0.1512542	total: 5m 29s	remaining: 8m 28s
472:	learn: 0.1511371	total: 5m 30s	remaining: 8m 28s
473:	learn: 0.1510732	total: 5m 31s	remaining: 8m 27s
474:	learn: 0.1509346	total:

609:	learn: 0.1326909	total: 6m 57s	remaining: 6m 43s
610:	learn: 0.1326169	total: 6m 57s	remaining: 6m 42s
611:	learn: 0.1325932	total: 6m 58s	remaining: 6m 42s
612:	learn: 0.1324169	total: 6m 59s	remaining: 6m 41s
613:	learn: 0.1323171	total: 6m 59s	remaining: 6m 40s
614:	learn: 0.1321588	total: 7m	remaining: 6m 40s
615:	learn: 0.1320436	total: 7m 1s	remaining: 6m 39s
616:	learn: 0.1320050	total: 7m 1s	remaining: 6m 38s
617:	learn: 0.1318758	total: 7m 2s	remaining: 6m 37s
618:	learn: 0.1317958	total: 7m 2s	remaining: 6m 36s
619:	learn: 0.1317066	total: 7m 3s	remaining: 6m 36s
620:	learn: 0.1315808	total: 7m 4s	remaining: 6m 35s
621:	learn: 0.1314873	total: 7m 4s	remaining: 6m 34s
622:	learn: 0.1313781	total: 7m 5s	remaining: 6m 33s
623:	learn: 0.1313732	total: 7m 5s	remaining: 6m 33s
624:	learn: 0.1312592	total: 7m 6s	remaining: 6m 32s
625:	learn: 0.1312287	total: 7m 6s	remaining: 6m 31s
626:	learn: 0.1312046	total: 7m 7s	remaining: 6m 30s
627:	learn: 0.1310756	total: 7m 8s	remaining

763:	learn: 0.1153988	total: 8m 32s	remaining: 4m 52s
764:	learn: 0.1153311	total: 8m 32s	remaining: 4m 51s
765:	learn: 0.1152860	total: 8m 33s	remaining: 4m 50s
766:	learn: 0.1152414	total: 8m 33s	remaining: 4m 50s
767:	learn: 0.1152227	total: 8m 34s	remaining: 4m 49s
768:	learn: 0.1151039	total: 8m 34s	remaining: 4m 48s
769:	learn: 0.1150513	total: 8m 35s	remaining: 4m 47s
770:	learn: 0.1149068	total: 8m 36s	remaining: 4m 47s
771:	learn: 0.1147777	total: 8m 36s	remaining: 4m 46s
772:	learn: 0.1146918	total: 8m 37s	remaining: 4m 45s
773:	learn: 0.1145533	total: 8m 37s	remaining: 4m 44s
774:	learn: 0.1145016	total: 8m 38s	remaining: 4m 44s
775:	learn: 0.1143746	total: 8m 38s	remaining: 4m 43s
776:	learn: 0.1143632	total: 8m 39s	remaining: 4m 42s
777:	learn: 0.1142217	total: 8m 40s	remaining: 4m 42s
778:	learn: 0.1141238	total: 8m 40s	remaining: 4m 41s
779:	learn: 0.1140044	total: 8m 41s	remaining: 4m 40s
780:	learn: 0.1139193	total: 8m 41s	remaining: 4m 39s
781:	learn: 0.1138582	total:

916:	learn: 0.1022756	total: 9m 59s	remaining: 3m 5s
917:	learn: 0.1021880	total: 10m	remaining: 3m 4s
918:	learn: 0.1021405	total: 10m	remaining: 3m 3s
919:	learn: 0.1020970	total: 10m 1s	remaining: 3m 2s
920:	learn: 0.1020913	total: 10m 1s	remaining: 3m 2s
921:	learn: 0.1020228	total: 10m 2s	remaining: 3m 1s
922:	learn: 0.1019473	total: 10m 2s	remaining: 3m
923:	learn: 0.1017420	total: 10m 3s	remaining: 3m
924:	learn: 0.1017104	total: 10m 4s	remaining: 2m 59s
925:	learn: 0.1016234	total: 10m 4s	remaining: 2m 58s
926:	learn: 0.1014513	total: 10m 5s	remaining: 2m 58s
927:	learn: 0.1012741	total: 10m 6s	remaining: 2m 57s
928:	learn: 0.1012545	total: 10m 6s	remaining: 2m 57s
929:	learn: 0.1011419	total: 10m 7s	remaining: 2m 56s
930:	learn: 0.1011226	total: 10m 7s	remaining: 2m 55s
931:	learn: 0.1009704	total: 10m 8s	remaining: 2m 54s
932:	learn: 0.1009184	total: 10m 9s	remaining: 2m 54s
933:	learn: 0.1007972	total: 10m 9s	remaining: 2m 53s
934:	learn: 0.1007230	total: 10m 10s	remaining: 

1066:	learn: 0.0924828	total: 11m 25s	remaining: 1m 25s
1067:	learn: 0.0924010	total: 11m 26s	remaining: 1m 24s
1068:	learn: 0.0922707	total: 11m 26s	remaining: 1m 24s
1069:	learn: 0.0922047	total: 11m 27s	remaining: 1m 23s
1070:	learn: 0.0921778	total: 11m 27s	remaining: 1m 22s
1071:	learn: 0.0921529	total: 11m 28s	remaining: 1m 22s
1072:	learn: 0.0921386	total: 11m 28s	remaining: 1m 21s
1073:	learn: 0.0921025	total: 11m 29s	remaining: 1m 20s
1074:	learn: 0.0920809	total: 11m 30s	remaining: 1m 20s
1075:	learn: 0.0919739	total: 11m 30s	remaining: 1m 19s
1076:	learn: 0.0919093	total: 11m 31s	remaining: 1m 18s
1077:	learn: 0.0918203	total: 11m 31s	remaining: 1m 18s
1078:	learn: 0.0917568	total: 11m 32s	remaining: 1m 17s
1079:	learn: 0.0916511	total: 11m 32s	remaining: 1m 16s
1080:	learn: 0.0915273	total: 11m 33s	remaining: 1m 16s
1081:	learn: 0.0914620	total: 11m 34s	remaining: 1m 15s
1082:	learn: 0.0913598	total: 11m 34s	remaining: 1m 15s
1083:	learn: 0.0913537	total: 11m 35s	remaining:

In [22]:
a = catboost_1.predict(test_)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_5.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [23]:
catboost_1.save_model("model_3_.cbm")
# catboost_1=CatBoostRegressor(n_estimators=1200,max_depth=11)
# catboost_1.load_model("model_1_.cbm")

In [25]:
import catboost
imp = catboost_1.get_feature_importance(catboost.Pool(train_, y_))
feat = list(data_train_2.drop([ "TARGET","BIRTHDATE",'WORKERSRANGE_1','id_contract']+date_drop, axis=1).columns)
sort = [(imp[i],feat[i]) for i in range(len(imp))]
sorted_by_0 = sorted(sort, key=lambda tup: tup[0], reverse=True)
bad_f = []
for i in range(len(imp)):
    print(sorted_by_0[i][1], round(sorted_by_0[i][0], 2))

is_null_n1 5.39
OKVED_CODE2 4.37
is_null_n-1 3.93
OKVED_CODE1 3.79
sign-first 3.36
OKVED_CODE 3.17
DATEFIRSTREG 3.12
IP_n1 2.45
id_client 2.36
OKTMO_CODE 2.11
NOTADMITTEDNUMBEREVER-ADMITTEDNUMBER_95_EVER 2.07
IP_n-1 2.05
WORKERSRANGE 1.91
sign-taxreg 1.7
TELEPHONECOUNT 1.64
SIGN_DATE_date_n1 1.49
sign-taxregpay 1.46
SIGN_DATE_month 1.34
WORKERSRANGE1 1.3
SIGN_DATE_date_n-1 1.29
TAXREG_REGDATEmonth 1.23
TAXREG_REGDATEday 1.22
SUM_95_YEAR_n-1 1.2
DATEFIRSTREGday 1.17
SIGN_DATE 1.14
SUM_F 1.05
DATEFIRSTREGmonth 0.99
F2120 0.99
OKATO_FED 0.95
SUM_95_YEAR_n1 0.94
OKTMO_FED 0.88
SIGN_DATE_date_n-3 0.87
OKATO_REGIONCODE 0.87
F1250 0.83
F2110_diff 0.83
TAXREG_REGDATE 0.82
WINNERNUMBER_95_EVER 0.8
WORKERSRANGE2 0.78
NOTADMITTEDNUMBER_95_YEAR 0.75
taxregpay-taxreg 0.74
F1300 0.65
SIGN_DATE_date_n3 0.63
F1210 0.62
SIGNEDNUMBER_95_EVER 0.61
F2110 0.6
taxregpay-first 0.59
AGE 0.57
F1230_diff 0.56
F1230 0.55
is_null_110- 0.54
F1510 0.54
MANAGERCOUNTINCOUNTRY 0.52
TAXREGPAY_REGDATEmonth 0.52
is_null 

In [26]:
ind_mean=list(data_train_2.groupby("id_client").mean()['TARGET'].index)
mean_=data_train_2.groupby("id_client").mean()['TARGET']
ind_count=list(data_train_2.groupby("id_client").count()['TARGET'].index)
count_=data_train_2.groupby("id_client").count()['TARGET']
ind_sum=list(data_train_2.groupby("id_client").sum()['TARGET'].index)
sum_=data_train_2.groupby("id_client").sum()['TARGET']
list_cl_tr = list(data_train_2['id_client'])
list_cl_t = list(data_test_2['id_client'])
list_cl_tr_m = []
list_cl_tr_c = []
list_cl_tr_s = []
list_cl_t_m = []
list_cl_t_c = []
list_cl_t_s = []
for i in list_cl_tr:
#     print(i)
    if count_[i] ==1:
        list_cl_tr_m.append(0)
        list_cl_tr_c.append(0)
        list_cl_tr_s.append(0)
    else:
        list_cl_tr_m.append(mean_[i])
        list_cl_tr_c.append(count_[i])
        list_cl_tr_s.append(sum_[i])
for i in list_cl_t:
#     print(i)
    if i in list_cl_tr:
        list_cl_t_m.append(mean_[i])
        list_cl_t_c.append(count_[i])
        list_cl_t_s.append(sum_[i])
    else:
        list_cl_t_m.append(-1)
        list_cl_t_c.append(0)
        list_cl_t_s.append(0)
data_train_2['mean'] = list_cl_tr_m
data_test_2['mean'] = list_cl_t_m
data_train_2['count'] = list_cl_tr_c
data_test_2['count'] = list_cl_t_c
data_train_2['sum'] = list_cl_tr_s
data_test_2['sum'] = list_cl_t_s

In [27]:
pred_stat1= list(pd.read_csv('./subm_5.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat2= list_cl_t_m
res = []
c = 0
for i in range(len(pred_stat1)):
    if pred_stat2[i] ==-1:
        res.append(pred_stat1[i])
    else:
        c+=1
#         print(pred_stat2[i],pred_stat1[i])
        if pred_stat2[i]>0:
            res.append(pred_stat1[i]+0.07*pred_stat2[i])
        if pred_stat2[i]==0:
            res.append(pred_stat1[i]-0.07)
a = np.array(res)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_6.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
from sklearn.base import BaseEstimator, ClassifierMixin
import lightgbm as lgb
import xgboost as xgb
# model1 = lgb.LGBMClassifier(num_leaves=5, learning_rate=0.05, n_estimators=int(600*2),
#                                           colsample_bytree=0.5, subsample=0.5,
#                                           nthread=-1, random_state=3)
model1 = lgb.LGBMClassifier(num_leaves=40,
                               learning_rate=0.05,
                               n_estimators=400)

In [33]:
model1.fit(train_,y_)

LGBMClassifier(learning_rate=0.05, n_estimators=400, num_leaves=40)

In [34]:
a = model1.predict_proba(test_)[:, 1]
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_7.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
ind_mean=list(data_train_2.groupby("id_client").mean()['TARGET'].index)
mean_=data_train_2.groupby("id_client").mean()['TARGET']
ind_count=list(data_train_2.groupby("id_client").count()['TARGET'].index)
count_=data_train_2.groupby("id_client").count()['TARGET']
ind_sum=list(data_train_2.groupby("id_client").sum()['TARGET'].index)
sum_=data_train_2.groupby("id_client").sum()['TARGET']
list_cl_tr = list(data_train_2['id_client'])
list_cl_t = list(data_test_2['id_client'])
list_cl_tr_m = []
list_cl_tr_c = []
list_cl_tr_s = []
list_cl_t_m = []
list_cl_t_c = []
list_cl_t_s = []
for i in list_cl_tr:
#     print(i)
    if count_[i] ==1:
        list_cl_tr_m.append(0)
        list_cl_tr_c.append(0)
        list_cl_tr_s.append(0)
    else:
        list_cl_tr_m.append(mean_[i])
        list_cl_tr_c.append(count_[i])
        list_cl_tr_s.append(sum_[i])
for i in list_cl_t:
#     print(i)
    if i in list_cl_tr:
        list_cl_t_m.append(mean_[i])
        list_cl_t_c.append(count_[i])
        list_cl_t_s.append(sum_[i])
    else:
        list_cl_t_m.append(-1)
        list_cl_t_c.append(0)
        list_cl_t_s.append(0)
data_train_2['mean'] = list_cl_tr_m
data_test_2['mean'] = list_cl_t_m
data_train_2['count'] = list_cl_tr_c
data_test_2['count'] = list_cl_t_c
data_train_2['sum'] = list_cl_tr_s
data_test_2['sum'] = list_cl_t_s

In [35]:
pred_stat1= list(pd.read_csv('./subm_7.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat2= list_cl_t_m
res = []
c = 0
for i in range(len(pred_stat1)):
    if pred_stat2[i] ==-1:
        res.append(pred_stat1[i])
    else:
        c+=1
#         print(pred_stat2[i],pred_stat1[i])
        if pred_stat2[i]>0:
            res.append(pred_stat1[i]+0.07*pred_stat2[i])
        if pred_stat2[i]==0:
            res.append(pred_stat1[i]-0.07)
a = np.array(res)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_8.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [46]:
pred_stat1= list(pd.read_csv('./subm_8.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat2= list(pd.read_csv('./subm_6.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat3= list(pd.read_csv('./subm_4.csv', sep=';', encoding='cp1251')["TARGET"])

# pred_stat2= list_cl_t_m
res = []
c = 0
for i in range(len(pred_stat1)):
    res.append(pred_stat1[i]*0.33+0.33*pred_stat2[i]+pred_stat3[i]*0.33)
a = np.array(res)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_12.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
pred_stat1= list(pd.read_csv('./0.9251.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat2= list(pd.read_csv('./subm_12.csv', sep=';', encoding='cp1251')["TARGET"])
# pred_stat3= list(pd.read_csv('./subm_4.csv', sep=';', encoding='cp1251')["TARGET"])

# pred_stat2= list_cl_t_m
res = []
c = 0
for i in range(len(pred_stat1)):
    res.append(pred_stat1[i]*0.5+0.5*pred_stat2[i])
a = np.array(res)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_13.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
pred_stat1= list(pd.read_csv('./new_925.csv', sep=';', encoding='cp1251')["TARGET"])
pred_stat2= list(pd.read_csv('./subm_13.csv', sep=';', encoding='cp1251')["TARGET"])
# pred_stat3= list(pd.read_csv('./subm_4.csv', sep=';', encoding='cp1251')["TARGET"])

# pred_stat2= list_cl_t_m
res = []
c = 0
for i in range(len(pred_stat1)):
    res.append(pred_stat1[i]*0.3+0.7*pred_stat2[i])
a = np.array(res)
a = a - min(a)
a = a / max(a)
df=data_test_2[['id_contract']]
df['a'] = a
## 
pd.DataFrame({'id_contract': df.id_contract, 'TARGET': df.a}).to_csv('subm_14.csv', index=False,sep=';')

c:\users\dolga\opencv\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
